# 식신 - 제주시청

In [34]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup



### 데이터 가져오기(해보기 전에 긁어지는지 아닌지 알수없다)

In [35]:
site = 'https://www.siksinhot.com/'
url = f'{site}search?keywords={quote("제주시청")}'
req = requests.get(url)
html=req.text


In [36]:
soup = BeautifulSoup(html, 'html.parser')

### 찾으려고 하는 데이터의 태그 찾기

In [58]:
#모든 사진이 출력된다
lis = soup.select_one('div.listTy1').find_all('li')
len(lis)

60

In [38]:
# 해당페이지의 사진만 찾으려면 아래와 같은 방법으로
lis=soup.select('div.listTy1 > ul > li')  #부등호 방향으로 자식 seletor 표현
len(lis)

12

### 하나를 선택해서 원하는 정보 추출
- 상호, 평점, 주요메뉴, 서브페이지의 url
- 전화번호, 주소(서브페이지)

In [39]:
li=lis[1]
href = li.find('a')['href']
href

'/P/466268'

In [40]:
score = li.select_one('.score').string
score

'3.4'

In [41]:
store = li.select_one('.store').get_text()
store

'안양돈까스 제주시점'

In [42]:
# p 태그를 찾으려면 find() 쓸것
menu = li.find('p').get_text()
menu

'돈까스 세트, 치즈돈까스 세트'

In [43]:
sub_url = site + href
sub_url

'https://www.siksinhot.com//P/466268'

In [45]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [46]:
tel = sub_soup.select_one('.p_tel').find('p').get_text()
tel

'064-799-1434'

In [47]:
addr = sub_soup.select_one('.txt_adr').get_text()
addr

'제주특별자치도 제주시 애월읍 애월로 40'

### 페이지에 있는 데이터 가져오기

In [48]:
# try, except로 에러가 어디서 발생했는지 잡아낸다.

lines=[]
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one('.score').string
        store = li.select_one('.store').get_text()
        menu = li.find('p').get_text()
        
        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    except : 
        print(index)

    lines.append([store, score, menu, tel, addr])

0


### 에러난 부분을 처리할 수 있도록 수정

In [50]:
li = lis[0]
score = li.select_one('.cnt>em').get_text()
score

'평가중'

In [51]:
lis[1].select_one('.cnt>em').get_text()

'3.4'

### 에러가 해결될 때 까지 5,6을 반복

In [52]:
lines=[]
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one('.cnt>em').get_text()
        if score =='평가중':
            score = '0.0'
        store = li.select_one('.store').get_text()
        menu = li.find('p').get_text()
        
        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    except : 
        print(index)

    lines.append([store, score, menu, tel, addr])

### 데이터프레임으로 만들기

In [57]:
df=pd.DataFrame(lines, columns=['업소명', '평점', '메뉴','전화번호','주소'])
df.set_index('업소명', inplace=True)
df.head()


,평점,메뉴,전화번호,주소
업소명,,,,
제주시새우리,0.0,"딱새우김밥, 간장새우김밥, 양념새우컵밥, 딱새우꼬막무침, 한라봉에이드",064-900-2527,제주특별자치도 제주시 무근성7길 21
안양돈까스 제주시점,3.4,"돈까스 세트, 치즈돈까스 세트",064-799-1434,济州特别自治道济州市涯月邑로
올래국수,3.8,고기국수,064-742-7355,제주특별자치도 제주시 귀아랑길 24
명진전복,3.8,"전복죽, 전복돌솥밥, 전복구이, 전복회",064-782-9944,제주특별자치도 제주시 구좌읍 해맞이해안로 1282
삼성혈해물탕,3.9,"해물탕(특대), 해물탕(대), 해물탕(중), 해물탕(소), 전복회(소)",064-745-3000,제주특별자치도 제주시 선덕로5길 20
